In [ ]:
import tensorflow as tf
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import os

In [ ]:
train_dir = 'face2/train'
validation_dir = 'face2/validation'
test_dir='face2/test'


train_jaydev_dir = os.path.join(train_dir, 'jaydev')
train_rohan_dir = os.path.join(train_dir, 'rohan')


validation_jaydev_dir = os.path.join(validation_dir, 'jaydev')
validation_rohan_dir = os.path.join(validation_dir, 'rohan')

test_jaydev_dir = os.path.join(test_dir, 'jaydev')
test_rohan_dir = os.path.join(test_dir, 'rohan')

train_jaydev_dir_fnames = os.listdir( train_jaydev_dir)


print(train_jaydev_dir_fnames[:10])

In [ ]:
model = tf.keras.models.Sequential([
    # input shape is the desired size of the image 150x150 with 3 bytes color
    #  first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #  second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #  fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #  fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten 
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('jaydev') and 1 for the other ('rohan')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_datagen  = ImageDataGenerator( rescale = 1.0/255. )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    #batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     

validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                         #batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              
                              epochs=10,
                             
                              verbose=1)

In [ ]:
import numpy as np
from keras.preprocessing import image

import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0)

while(True):
    return_value, image = camera.read()
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        face=image[y:y+h,x:x+w]
        face=cv2.resize(face,(150,150))
        #cv2.imwrite('try/try'+str(i)+'.jpg', face)# here specify ur folder
        image=cv2.rectangle(image, (x,y), (x+w,y+h), (255,0,0), 3)
        
    
    #path='try/try'+str(i)+'.jpg' 
    #img=cv2.imread(path)
  
        z=np.array(face)
        z=np.expand_dims(z, axis=0)
        images = np.vstack([z])

        classes = model.predict(images, batch_size=10)

        print(classes[0])

        if classes[0]>0:
            image=cv2.putText(image,'rohan',(x+60,y-10),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),2)
            print(" is a rohan")
            
        else:
            image=cv2.putText(image,'jaydev',(x+60,y-10),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),2)
            print(" is a jaydev")
        
    cv2.imshow('photu',image)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

del(camera)
cv2.waitKey(0)
cv2.destroyAllWindows()